In [1]:
import os, json

import pandas as pd
from langchain_openai import AzureChatOpenAI
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from PIL import Image, ImageDraw, ImageFont

In [2]:
df_scenarios = pd.read_csv("df_scenarios.csv")
df_claims = pd.read_csv("df_claims.csv")

In [3]:
creds_path = r"C:\Pranav\azure_openai_creds.json"

with open(creds_path, "r") as f:
    creds = json.load(f)

llm = AzureChatOpenAI(
    azure_endpoint=creds["azure_endpoint"],
    api_key=creds["OPENAI_API_KEY"],
    api_version=creds["OPENAI_API_VERSION"],
    deployment_name=creds["deployment_name"]
)

In [4]:
df_scenarios.head()

scenario                                        description  \
0     Valid Vehicle  A policyholder submits a claim for vehicle dam...   
1  Missing Document  A policyholder submits a claim for storm damag...   
2    Duplicate Bill  A policyholder submits a claim for hospital st...   
3   Mid-level Fraud  A policyholder submits a claim for a car accid...   
4     Valid Medical  A policyholder submits a claim for a medically...   

                              claim_type  \
0    Auto Insurance (Collision Coverage)   
1     Homeowners Insurance (HO-3 Policy)   
2  Health Insurance (Major Medical Plan)   
3    Auto Insurance (Collision Coverage)   
4  Health Insurance (Major Medical Plan)   

                                       covered_rules  
0  {'document_verification_agent': ['police_repor...  
1  {'document_verification_agent': ['proof_of_los...  
2  {'document_verification_agent': ['submitted_do...  
3  {'document_verification_agent': ['police_repor...  
4  {'document_verification_agent': ['submitted_do...

In [5]:
df_claims.head()

name policy_id policy_start  policy_end  \
0               Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1               Valid Medical  HLT-2001   2025-01-01  2025-12-31   
2        Stolen Vehicle Claim  VHC-1002   2025-01-01  2025-12-31   
3          Water Damage Claim  HOM-3002   2025-01-01  2025-12-31   
4  Outpatient Procedure Claim  HLT-2002   2025-01-01  2025-12-31   

                                claim_type accident_date  claim_amount  \
0      Auto Insurance (Collision Coverage)    2025-06-20          5000   
1    Health Insurance (Major Medical Plan)    2025-05-15         12000   
2  Auto Insurance (Comprehensive Coverage)    2025-06-05         25000   
3       Homeowners Insurance (HO-3 Policy)    2025-03-12          6000   
4    Health Insurance (Major Medical Plan)    2025-06-10          3000   

                         document  is_valid_claim  \
0         repair_bill_vehicle.pdf            True   
1  hospital_bill_knee_surgery.pdf            True   
2         police_report_theft.pdf            True   
3              plumber_report.pdf            True   
4            colonoscopy_bill.pdf            True   

                                              reason  
0  All required documents provided, policy active...  
1  Medically necessary procedure, in-network prov...  
2  Theft covered under comprehensive terms, polic...  
3  Burst pipe covered, complete documentation inc...  
4  Medically necessary procedure, in-network prov...

In [6]:
# df_claims["document"].to_list()

In [7]:
# for ind, row in df_claims.iterrows():
#     print(row["document"], "\n")
#     # print(row.to_dict())
#     # break

In [8]:
output_dir = r"C:\Pranav\claim_documents"
os.makedirs(output_dir, exist_ok=True)

In [9]:
def generate_pdf_content(row):
    """Generate structured content for the bill using LLM."""
    # Prepare row data as a string of key-value pairs, excluding 'document', 'reason', and 'is_valid_claim'
    row_data = {k: v for k, v in row.items() if k not in ['document', 'reason', 'is_valid_claim']}
    row_data_str = '\n'.join([f"- {k}: {v}" for k, v in row_data.items() if v is not None])
    
    prompt = f"""
    Create a structured description for a bill based on the following details:
    {row_data_str}

    Format the response as a list of strings, where each string is a key-value pair in the format 'Key: Value', e.g., ['Name: Valid Vehicle', 'Policy ID: VHC-1001', ...].
    Include a header line: 'Bill for {row.get('name', 'Customer')} - Policy {row.get('policy_id', 'Unknown')}'.
    Only include fields provided in the input, and do not add any extra fields or assumptions.
    Return only the list of strings, no explanations.
    """
    
    response = llm.invoke(prompt)
    content = response.content
    
    # Parse content if it's a string representation of a list
    try:
        import ast
        content_list = ast.literal_eval(content)
        if not isinstance(content_list, list):
            raise ValueError
        return content_list
    except (ValueError, SyntaxError):
        # Fallback: dynamically generate content from row
        content_list = [f"Bill for {row.get('name', 'Customer')} - Policy {row.get('policy_id', 'Unknown')}"]
        for key, value in row_data.items():
            if value is not None:
                content_list.append(f"{key.replace('_', ' ').title()}: {value}")
        return content_list

def create_pdf_bill(content_list, output_filename):
    """Create a PDF bill using ReportLab."""
    pdf_path = os.path.join(output_dir, output_filename)
    doc = SimpleDocTemplate(pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Define custom styles
    header_style = ParagraphStyle(
        name='Header',
        fontSize=16,
        leading=20,
        alignment=1,  # Center
        spaceAfter=20,
        fontName='Helvetica-Bold'
    )
    normal_style = styles['Normal']
    
    # Build document elements
    elements = []
    
    # Add header
    elements.append(Paragraph(content_list[0], header_style))
    elements.append(Spacer(1, 12))
    
    # Create table for bill details
    table_data = [[Paragraph(line, normal_style)] for line in content_list[1:]]
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, -1), colors.white),
        ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 12),
        ('LEFTPADDING', (0, 0), (-1, -1), 0),
        ('RIGHTPADDING', (0, 0), (-1, -1), 0),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(table)
    
    # Build PDF
    try:
        doc.build(elements)
        print(f"PDF saved to {pdf_path}")
        return pdf_path
    except Exception as e:
        print(f"Error generating PDF: {e}")
        return None

def generate_png_repair_bill(row, content_list, output_filename):
    """Generate a PNG bill using Pillow."""
    img_width, img_height = 600, 800
    img = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(img)
    
    # Try to use a TrueType font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 20)
        font_small = ImageFont.truetype("arial.ttf", 16)
    except IOError:
        font = ImageFont.load_default()
        font_small = ImageFont.load_default()
    
    # Draw header
    header = content_list[0]
    draw.text((img_width // 2, 30), header, fill='black', font=font, anchor='mm')
    
    # Draw bill details
    y_offset = 100
    for line in content_list[1:]:
        draw.text((30, y_offset), line, fill='black', font=font_small, anchor='lm')
        y_offset += 30
    
    # Save PNG
    png_path = os.path.join(output_dir, output_filename)
    try:
        img.save(png_path, 'PNG')
        print(f"PNG saved to {png_path}")
        return png_path
    except Exception as e:
        print(f"Error generating PNG: {e}")
        return None

def process_document(row):
    """Process a single claim row and generate the document."""
    document = row['document']
    extension = os.path.splitext(document)[1].lower()
    
    # Generate content for both PDF and PNG
    content_list = generate_pdf_content(row)
    
    if extension == '.pdf':
        return create_pdf_bill(content_list, document)
    elif extension in ['.png', '.jpg', '.jpeg']:
        return generate_png_repair_bill(row, content_list, document)
    else:
        print(f"Unsupported file extension: {extension}")
        return None

In [12]:
df_claims["document_path"] = None

In [16]:
# df_claims

In [17]:
for ind, row in df_claims.iterrows():
    
    # print(row.to_dict())
    saved_document_path = process_document(row)
    df_claims.loc[ind, "document_path"] = saved_document_path

PDF saved to C:\Pranav\claim_documents\repair_bill_vehicle.pdf
PDF saved to C:\Pranav\claim_documents\hospital_bill_knee_surgery.pdf
PDF saved to C:\Pranav\claim_documents\police_report_theft.pdf
PDF saved to C:\Pranav\claim_documents\plumber_report.pdf
PDF saved to C:\Pranav\claim_documents\colonoscopy_bill.pdf
PDF saved to C:\Pranav\claim_documents\medical_emergency_letter.pdf
PDF saved to C:\Pranav\claim_documents\police_report_vandalism.pdf
PNG saved to C:\Pranav\claim_documents\roof_repair_estimate.png
PDF saved to C:\Pranav\claim_documents\incident_report.pdf
PDF saved to C:\Pranav\claim_documents\death_certificate.pdf
PNG saved to C:\Pranav\claim_documents\repair_estimate.png
PDF saved to C:\Pranav\claim_documents\hospital_bill_duplicate.pdf
PDF saved to C:\Pranav\claim_documents\repair_estimate_exaggerated.pdf
PDF saved to C:\Pranav\claim_documents\altered_death_certificate.pdf
PDF saved to C:\Pranav\claim_documents\fire_department_report.pdf
PDF saved to C:\Pranav\claim_docume

In [ ]:
# df_claims.to_csv("df_claims_with_document_path.csv", index=False)

In [ ]:
# df_claims